In [4]:
def set_cfg():
    import os
    import json
    f = open("../api/kaggle.json", 'r')
    json_data = json.load(f)
    os.environ['KAGGLE_USERNAME'] = json_data['username']
    os.environ['KAGGLE_KEY'] = json_data['key']

# def submit():
#     !pip install kaggle
#     
def check_submissions():
    !kaggle competitions submissions -c open-problems-multimodal

In [5]:
set_cfg()

In [6]:
!pip install kaggle
!kaggle kernels list

  Using cached kaggle-1.5.12-py3-none-any.whl
ref                                                              title                                               author                lastRunTime          totalVotes  
---------------------------------------------------------------  --------------------------------------------------  --------------------  -------------------  ----------  
wouldyoujustfocus/l-king-for-a-ds-job                            L👀king for a DS job ??                              kaoutar               2022-11-15 07:05:32          22  
gautamchettiar/data-preprocessing-eda-and-classifications        Data Preprocessing, EDA, and Classifications        Gautam Chettiar       2022-11-15 05:13:21           4  
saramah/bank-loan-classification                                 Bank_Loan_classification                            Sara Mahdavi          2022-11-15 07:14:45          29  
saramah/mobile-price-prediction-classification                   Mobile_Price_Prediction_

In [7]:
import os, gc, pickle, warnings
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from colorama import Fore, Back, Style
from matplotlib.ticker import MaxNLocator
import seaborn as sns
import os,gc,time,random
from tqdm.notebook import tqdm

from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.model_selection import KFold
from sklearn.preprocessing import StandardScaler, scale
from sklearn.decomposition import PCA, TruncatedSVD
from sklearn.dummy import DummyRegressor
from sklearn.pipeline import make_pipeline, Pipeline
from sklearn.linear_model import Ridge, LinearRegression, Lasso
from sklearn.metrics import mean_squared_error

import scipy
import scipy.sparse

warnings.filterwarnings('ignore')

In [8]:
FP_INPUT_DIR = '../input/'
FP_OUTPUT_DIR = './'


FP_SPARSE_DATA = os.path.join(FP_INPUT_DIR, "multimodal-single-cell-as-sparse-matrix/")
FP_PCA_DATA = os.path.join(FP_INPUT_DIR, "single-cell-multi-svd-pkl/") 

FP_CELL_METADATA = os.path.join(FP_SPARSE_DATA,"metadata.parquet")

FP_CITE_TRAIN_INPUTS_VALUES = os.path.join(FP_SPARSE_DATA,"train_cite_inputs_values.sparse.npz")
FP_CITE_TRAIN_INPUTS_IDXCOL = os.path.join(FP_SPARSE_DATA,"train_cite_inputs_idxcol.npz")
FP_CITE_TRAIN_TARGETS_VALUES = os.path.join(FP_SPARSE_DATA,"train_cite_targets_values.sparse.npz")
FP_CITE_TRAIN_TARGETS_IDXCOL = os.path.join(FP_SPARSE_DATA,"train_cite_targets_idxcol.npz")
FP_CITE_TEST_INPUTS_VALUES = os.path.join(FP_SPARSE_DATA,"test_cite_inputs_values.sparse.npz")
FP_CITE_TEST_INPUTS_IDXCOL = os.path.join(FP_SPARSE_DATA,"test_cite_inputs_idxcol.npz")

FP_MULTI_TRAIN_INPUTS_VALUES = os.path.join(FP_SPARSE_DATA,"train_multi_inputs_values.sparse.npz")
FP_MULTI_TRAIN_INPUTS_IDXCOL = os.path.join(FP_SPARSE_DATA,"train_multi_inputs_idxcol.npz")
FP_MULTI_TRAIN_TARGETS_VALUES = os.path.join(FP_SPARSE_DATA,"train_multi_targets_values.sparse.npz")
FP_MULTI_TRAIN_TARGETS_IDXCOL = os.path.join(FP_SPARSE_DATA,"train_multi_targets_idxcol.npz")
FP_MULTI_TEST_INPUTS_VALUES = os.path.join(FP_SPARSE_DATA,"test_multi_inputs_values.sparse.npz")
FP_MULTI_TEST_INPUTS_IDXCOL = os.path.join(FP_SPARSE_DATA,"test_multi_inputs_idxcol.npz")

FP_SUBMISSION = os.path.join(FP_SPARSE_DATA,"sample_submission.parquet")
FP_EVALUATION = os.path.join(FP_SPARSE_DATA,"evaluation.parquet")


In [9]:
# read sample submission
preds_cite = np.load('preds_cite/preds_cite_singlecat_11-14-17:48.npy')
preds_multi = np.load('preds_multi/preds_multi_single_cat.npy')

In [10]:
# citeseq
Y = scipy.sparse.load_npz(FP_CITE_TRAIN_TARGETS_VALUES)
# Copy the targets for the data leak but useless since the change in the public LB...
preds_cite[:7476] = Y[:7476].toarray()
del Y
gc.collect()

77

In [11]:
# multi
eval_ids = pd.read_parquet("../input/multimodal-single-cell-as-sparse-matrix/evaluation.parquet")
eval_ids.cell_id = eval_ids.cell_id.astype(pd.CategoricalDtype())
eval_ids.gene_id = eval_ids.gene_id.astype(pd.CategoricalDtype())

submission_multi = pd.Series(name='target',
                       index=pd.MultiIndex.from_frame(eval_ids), 
                       dtype=np.float32)
display(submission_multi)

y_columns = np.load("../input/multimodal-single-cell-as-sparse-matrix/train_multi_targets_idxcol.npz",
                   allow_pickle=True)["columns"]

test_index = np.load("../input/multimodal-single-cell-as-sparse-matrix/test_multi_inputs_idxcol.npz",
                    allow_pickle=True)["index"]

cell_dict = dict((k,v) for v,k in enumerate(test_index)) 
assert len(cell_dict)  == len(test_index)

gene_dict = dict((k,v) for v,k in enumerate(y_columns))
assert len(gene_dict) == len(y_columns)

eval_ids_cell_num = eval_ids.cell_id.apply(lambda x:cell_dict.get(x, -1))
eval_ids_gene_num = eval_ids.gene_id.apply(lambda x:gene_dict.get(x, -1))
valid_multi_rows = (eval_ids_gene_num !=-1) & (eval_ids_cell_num!=-1)

submission_multi.iloc[valid_multi_rows] = preds_multi[eval_ids_cell_num[valid_multi_rows].to_numpy(),
eval_ids_gene_num[valid_multi_rows].to_numpy()]

del eval_ids_cell_num, eval_ids_gene_num, valid_multi_rows, eval_ids, test_index, y_columns
gc.collect()

submission_multi.reset_index(drop=True, inplace=True)
submission_multi.index.name = 'row_id'

submission_multi

row_id    cell_id       gene_id        
0         c2150f55becb  CD86              NaN
1         c2150f55becb  CD274             NaN
2         c2150f55becb  CD270             NaN
3         c2150f55becb  CD155             NaN
4         c2150f55becb  CD112             NaN
                                           ..
65744175  2c53aa67933d  ENSG00000134419   NaN
65744176  2c53aa67933d  ENSG00000186862   NaN
65744177  2c53aa67933d  ENSG00000170959   NaN
65744178  2c53aa67933d  ENSG00000107874   NaN
65744179  2c53aa67933d  ENSG00000166012   NaN
Name: target, Length: 65744180, dtype: float32

row_id
0                NaN
1                NaN
2                NaN
3                NaN
4                NaN
              ...   
65744175    2.904297
65744176   -0.401123
65744177   -0.397461
65744178    0.165527
65744179    2.404297
Name: target, Length: 65744180, dtype: float32

In [12]:
submission = pd.read_csv('../input/open-problems-multimodal/sample_submission.csv',index_col='row_id', squeeze=True)
submission.iloc[:len(preds_cite.ravel())] = preds_cite.ravel()
submission.iloc[len(preds_cite.ravel()):] = submission_multi.iloc[len(preds_cite.ravel()):]
submission

row_id
0           1.167969
1           0.622559
2           0.106934
3           0.324951
4           3.332031
              ...   
65744175    2.904297
65744176   -0.401123
65744177   -0.397461
65744178    0.165527
65744179    2.404297
Name: target, Length: 65744180, dtype: float64

In [24]:
preds_cite_singlecat_retrain = np.load('preds_cite/preds_cite_singlecat_retrain_11-15-21:59.npy')
preds_cite_singlecat1 = np.load('preds_cite/preds_cite_singlecat_11-14-17:36.npy')
preds_cite_singlecat2 = np.load('preds_cite/preds_cite_singlecat_11-14-17:48.npy')
preds_cite_multicat = np.load('preds_cite/preds_cite_multicat.npy')
preds_cite_single_lgbm = np.load('preds_cite/preds_cite_lgbm_single.npy')


((48663, 140), 6812820)

In [43]:
def standardize(preds):
    i = 0
    for p in tqdm(preds):
        preds[i] = (p-np.mean(p))/np.std(p)
        i+=1
    return preds

In [44]:
preds_cite_singlecat_retrain = standardize(preds_cite_singlecat_retrain)
preds_cite_singlecat1 = standardize(preds_cite_singlecat1)
preds_cite_singlecat2 = standardize(preds_cite_singlecat2)
preds_cite_multicat = standardize(preds_cite_multicat)
preds_cite_single_lgbm = standardize(preds_cite_single_lgbm)

  0%|          | 0/48663 [00:00<?, ?it/s]

  0%|          | 0/48663 [00:00<?, ?it/s]

  0%|          | 0/48663 [00:00<?, ?it/s]

  0%|          | 0/48663 [00:00<?, ?it/s]

  0%|          | 0/48663 [00:00<?, ?it/s]

In [47]:
preds_cite_singlecat1, preds_cite_singlecat2, preds_cite_multicat, preds_cite_single_lgbm

(array([[-0.503  , -0.4805 , -0.3618 , ..., -0.4368 ,  0.1532 ,  0.4482 ],
        [-0.4883 , -0.4846 , -0.3604 , ..., -0.4216 ,  0.2942 ,  0.4773 ],
        [-0.541  , -0.4094 , -0.2532 , ..., -0.4946 ,  1.662  ,  0.571  ],
        ...,
        [-0.4226 , -0.08636, -0.2021 , ..., -0.3418 , -0.2876 ,  0.4768 ],
        [-0.6436 , -0.501  , -0.27   , ..., -0.5537 ,  2.312  ,  0.603  ],
        [-0.579  , -0.4634 , -0.2615 , ..., -0.521  ,  2.31   ,  0.636  ]],
       dtype=float16),
 array([[-0.512 , -0.489 , -0.366 , ..., -0.4385,  0.1444,  0.4531],
        [-0.5054, -0.4783, -0.361 , ..., -0.4263,  0.2917,  0.4915],
        [-0.535 , -0.4048, -0.2544, ..., -0.4705,  1.547 ,  0.529 ],
        ...,
        [-0.4297, -0.0693, -0.2125, ..., -0.3516, -0.1844,  0.472 ],
        [-0.637 , -0.49  , -0.2783, ..., -0.5522,  2.3   ,  0.575 ],
        [-0.6074, -0.4546, -0.2522, ..., -0.5244,  2.297 ,  0.595 ]],
       dtype=float16),
 array([[-0.515  , -0.4192 , -0.3599 , ..., -0.435  ,  0.1567 

In [45]:
preds_cite_ens = 0.3*preds_cite_singlecat1 + 0.3*preds_cite_singlecat2 + 0.1*preds_cite_multicat + 0.3*preds_cite_single_lgbm

In [ ]:
submission = pd.read_csv('submission_lolo_total_ensembling.csv',index_col='row_id', squeeze=True)
submission

row_id
0           0.182096
1           0.010811
2          -0.101281
3           1.442052
4           2.616124
              ...   
65744175    5.422715
65744176   -0.302524
65744177   -0.293852
65744178    0.755234
65744179    4.725446
Name: target, Length: 65744180, dtype: float64

In [46]:
submission.iloc[:len(preds_cite_ens.ravel())] = preds_cite_ens.ravel()
submission

row_id
0          -0.349609
1          -0.394287
2          -0.374512
3           0.598145
4           1.335938
              ...   
65744175    5.422715
65744176   -0.302524
65744177   -0.293852
65744178    0.755234
65744179    4.725446
Name: target, Length: 65744180, dtype: float64

In [23]:
%time submission.to_csv('submission_single_cat_retrain.csv',index='row_id')

CPU times: user 2min 11s, sys: 2.95 s, total: 2min 14s
Wall time: 2min 14s


In [49]:
%time preds_multi_single_cat = np.load('preds_multi/preds_multi_single_cat.npy')
%time preds_multi_single_lgbm = np.load('preds_multi/preds_multi_lgbm_single.npy')
preds_multi_single_cat,preds_multi_single_lgbm

CPU times: user 0 ns, sys: 1.07 s, total: 1.07 s
Wall time: 1.08 s
CPU times: user 163 ms, sys: 1.61 s, total: 1.77 s
Wall time: 18.5 s


(array([[-0.09296, -0.2954 , -0.317  , ...,  0.0268 ,  0.2224 ,  0.5923 ],
        [-0.1561 , -0.3267 , -0.2715 , ...,  0.1628 ,  0.2356 ,  0.626  ],
        [-0.1193 , -0.288  , -0.1714 , ...,  0.09045,  0.1656 ,  0.636  ],
        ...,
        [-0.2595 , -0.2693 , -0.2467 , ...,  0.1925 , -0.1059 ,  0.6787 ],
        [-0.277  , -0.2664 , -0.2803 , ...,  0.253  ,  0.08716,  0.9233 ],
        [-0.1078 , -0.1644 , -0.3118 , ...,  0.2844 ,  1.63   ,  1.234  ]],
       dtype=float16),
 array([[-0.1027 , -0.2988 , -0.325  , ...,  0.03497,  0.2211 ,  0.642  ],
        [-0.1484 , -0.3179 , -0.2952 , ...,  0.1724 ,  0.23   ,  0.618  ],
        [-0.11523, -0.2876 , -0.2167 , ...,  0.08075,  0.179  ,  0.6357 ],
        ...,
        [-0.26   , -0.2642 , -0.2324 , ...,  0.1492 , -0.09247,  0.674  ],
        [-0.2615 , -0.2563 , -0.2627 , ...,  0.255  ,  0.1384 ,  0.896  ],
        [-0.1115 , -0.1605 , -0.2896 , ...,  0.2976 ,  1.671  ,  1.28   ]],
       dtype=float16))

In [ ]:
preds_multi_single_cat = standardize(preds_multi_single_cat)
preds_multi_single_lgbm = standardize(preds_multi_single_lgbm)

  0%|          | 0/55935 [00:00<?, ?it/s]

  0%|          | 0/55935 [00:00<?, ?it/s]

In [51]:
preds_multi_ens = 0.5*preds_multi_single_cat + 0.5*preds_multi_single_lgbm
preds_multi_ens

array([[-0.1406 , -0.4272 , -0.4614 , ...,  0.04443,  0.3188 ,  0.8877 ],
       [-0.2107 , -0.446  , -0.392  , ...,  0.2319 ,  0.3223 ,  0.861  ],
       [-0.1759 , -0.4316 , -0.291  , ...,  0.1284 ,  0.2583 ,  0.954  ],
       ...,
       [-0.4014 , -0.412  , -0.3699 , ...,  0.2634 , -0.1531 ,  1.045  ],
       [-0.4    , -0.388  , -0.4028 , ...,  0.377  ,  0.1677 ,  1.351  ],
       [-0.1656 , -0.2454 , -0.454  , ...,  0.4395 ,  2.492  ,  1.898  ]],
      dtype=float16)

In [52]:
# multi
eval_ids = pd.read_parquet("../input/multimodal-single-cell-as-sparse-matrix/evaluation.parquet")
eval_ids.cell_id = eval_ids.cell_id.astype(pd.CategoricalDtype())
eval_ids.gene_id = eval_ids.gene_id.astype(pd.CategoricalDtype())

submission_multi = pd.Series(name='target',
                       index=pd.MultiIndex.from_frame(eval_ids), 
                       dtype=np.float32)
y_columns = np.load("../input/multimodal-single-cell-as-sparse-matrix/train_multi_targets_idxcol.npz",
                   allow_pickle=True)["columns"]

test_index = np.load("../input/multimodal-single-cell-as-sparse-matrix/test_multi_inputs_idxcol.npz",
                    allow_pickle=True)["index"]

cell_dict = dict((k,v) for v,k in enumerate(test_index)) 
assert len(cell_dict)  == len(test_index)

gene_dict = dict((k,v) for v,k in enumerate(y_columns))
assert len(gene_dict) == len(y_columns)

eval_ids_cell_num = eval_ids.cell_id.apply(lambda x:cell_dict.get(x, -1))
eval_ids_gene_num = eval_ids.gene_id.apply(lambda x:gene_dict.get(x, -1))
valid_multi_rows = (eval_ids_gene_num !=-1) & (eval_ids_cell_num!=-1)

submission_multi.iloc[valid_multi_rows] = preds_multi_ens[eval_ids_cell_num[valid_multi_rows].to_numpy(),
eval_ids_gene_num[valid_multi_rows].to_numpy()]

del eval_ids_cell_num, eval_ids_gene_num, valid_multi_rows, eval_ids, test_index, y_columns
gc.collect()

submission_multi.reset_index(drop=True, inplace=True)
submission_multi.index.name = 'row_id'

submission_multi

row_id
0                NaN
1                NaN
2                NaN
3                NaN
4                NaN
              ...   
65744175    4.429688
65744176   -0.625977
65744177   -0.621582
65744178    0.275391
65744179    3.773438
Name: target, Length: 65744180, dtype: float32

In [53]:
submission.iloc[len(preds_cite_ens.ravel()):] = submission_multi.iloc[len(preds_cite_ens.ravel()):]
submission

row_id
0          -0.349609
1          -0.394287
2          -0.374512
3           0.598145
4           1.335938
              ...   
65744175    4.429688
65744176   -0.625977
65744177   -0.621582
65744178    0.275391
65744179    3.773438
Name: target, Length: 65744180, dtype: float64

In [54]:
submission.to_csv('submission_GBDT_ens.csv',index='row_id')

In [56]:
sub32 = pd.read_csv('submission_x32y32.csv')
sub32

,row_id,target
0,0,0.275889
1,1,0.259389
2,2,0.356809
3,3,3.706386
4,4,4.317779
...,...,...
65744175,65744175,6.082031
65744176,65744176,0.036530
65744177,65744177,0.042511
65744178,65744178,1.142578


In [59]:
!kaggle datasets init -p submission_dataset

Data package template written to: submission_dataset/dataset-metadata.json


In [61]:
!kaggle datasets create -p submission_dataset

Skipping folder: .ipynb_checkpoints; use '--dir-mode' to upload folders
Starting upload for file submission_GBDT_ens.csv
100%|██████████████████████████████████████| 1.38G/1.38G [10:22<00:00, 2.37MB/s]
Upload successful: submission_GBDT_ens.csv (1GB)
Traceback (most recent call last):
  File "/opt/conda/envs/saturn/lib/python3.9/site-packages/urllib3/connectionpool.py", line 703, in urlopen
    httplib_response = self._make_request(
  File "/opt/conda/envs/saturn/lib/python3.9/site-packages/urllib3/connectionpool.py", line 449, in _make_request
    six.raise_from(e, None)
  File "<string>", line 3, in raise_from
  File "/opt/conda/envs/saturn/lib/python3.9/site-packages/urllib3/connectionpool.py", line 444, in _make_request
    httplib_response = conn.getresponse()
  File "/opt/conda/envs/saturn/lib/python3.9/http/client.py", line 1377, in getresponse
    response.begin()
  File "/opt/conda/envs/saturn/lib/python3.9/http/client.py", line 320, in begin
    version, status, reason = self